In [0]:
import numpy as np
#from torchvision import transforms
import spatial_transforms as transforms
import cv2
from objectAttentionModelConvLSTM import *
from attentionMapModel import attentionMap
from PIL import Image
import os
import imageio

In [3]:
DEVICE = "cuda"

####################Model definition###############################
num_classes = 61  # Classes in the pre-trained model
mem_size = 512
model_state_dict = '/content/drive/My Drive/Lorenzo/ego-rnn-two-in-one/results/16frames_two_in_one/rgb_16frames_two_in_one_3/model_rgb_state_dict.pth'  # Weights of the pre-trained model

model = attentionModel(num_classes=num_classes, mem_size=mem_size)
model.load_state_dict(torch.load(model_state_dict), strict=True)

model.train(False)
for params in model.parameters():
    params.requires_grad = False

model = model.to(DEVICE)
###################################################################

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth


/content/drive/My Drive/Lorenzo/ego-rnn-two-in-one/MyConvLSTMCell.py:31: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  torch.nn.init.xavier_normal(self.conv_i_xx.weight)
/content/drive/My Drive/Lorenzo/ego-rnn-two-in-one/MyConvLSTMCell.py:32: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  torch.nn.init.constant(self.conv_i_xx.bias, 0)
/content/drive/My Drive/Lorenzo/ego-rnn-two-in-one/MyConvLSTMCell.py:33: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  torch.nn.init.xavier_normal(self.conv_i_hh.weight)
/content/drive/My Drive/Lorenzo/ego-rnn-two-in-one/MyConvLSTMCell.py:35: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  torch.nn.init.xavier_normal(self.conv_f_xx.weight)
/content/drive/My Drive/Lorenzo/ego-rnn-two-in-one/MyConvLSTMCell.py:36: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  

In [0]:
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

preprocess = transforms.Compose([
    transforms.Scale(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize])

In [5]:
in_path = "/content/drive/My Drive/Lorenzo/ego-rnn-two-in-one/get_modulation/frames"
out_path = "/content/drive/My Drive/Lorenzo/ego-rnn-two-in-one/get_modulation/out"

inputFlow = []
inputFrame = []

frames = []
for i in range(1, len(os.listdir(in_path+"/rgb"))+1):
    print(i)

    # FLOW X
    fl_name = in_path + '/X/flow_x_' + str(int(round(i))).zfill(5) + '.png'  # zfill used to add leading zeros
    img = Image.open(fl_name)  # load single optical x frame
    inputFlow.append(preprocess(img.convert('L'), inv=False, flow=True))

    # FLOW Y
    fl_name = in_path + '/Y/flow_y_' + str(int(round(i))).zfill(5) + '.png'
    img = Image.open(fl_name)  # load single optical y frame
    inputFlow.append(preprocess(img.convert('L'), inv=False, flow=True))

    # RGB FRAME
    fl_name = in_path + '/rgb/' + 'rgb' + str(int(np.floor(i))).zfill(4) + ".png"
    img = Image.open(fl_name)  # load single rgb frame
    inputFrame.append(preprocess(img.convert('RGB')))


inputFlow = torch.stack(inputFlow, 0).squeeze(1)  # flow
inputFrame = torch.stack(inputFrame, 0)  # frame

inputFlow = inputFlow.unsqueeze(0)
inputFrame = inputFrame.unsqueeze(0)

inputFrame = Variable(inputFrame.permute(1, 0, 2, 3, 4).to(DEVICE))
    
inputFlow = inputFlow.view(
                    (inputFlow.shape[0], int(inputFlow.shape[1] / 2), 2, inputFlow.shape[2], inputFlow.shape[3]))
inputFlow = Variable(
                    inputFlow.permute(1, 0, 2, 3, 4).to(DEVICE))  # sequence length as first dimension


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [6]:
model((inputFrame, inputFlow), 3);

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [0]:
pre, post, beta, gamma = model.resNet.get_modulation_data()

In [0]:
def make_gif(path, out_path):
  kargs = {'duration': 0.05}                                                      # duration of single frame

  frames=[]
  for frame in sorted(os.listdir(path)):
    frames.append(imageio.imread(os.path.join(path, frame)))

  imageio.mimsave(out_path, frames, **kargs)                   # create gif

In [0]:
import matplotlib.pyplot as plt

os.makedirs(out_path+"/gif/")


In [13]:
for channel in range(64):
  path = out_path + f"/{channel}"
  os.makedirs(path)
  os.makedirs(path+"/pre")
  os.makedirs(path+"/post")
  os.makedirs(path+"/beta")
  os.makedirs(path+"/gamma")

  print(channel)
  for frame in range(20):
    plt.imsave(path + f"/pre/{frame}.png", pre[frame][0][channel], cmap="Greys")
    plt.imsave(path + f"/post/{frame}.png", post[frame][0][channel], cmap="Greys")
    plt.imsave(path + f"/beta/{frame}.png", beta[frame][0][channel], cmap="Greys")
    plt.imsave(path + f"/gamma/{frame}.png", gamma[frame][0][channel], cmap="Greys")


  make_gif(path + f"/pre/", out_path+f"/gif/{channel}_pre.gif")
  make_gif(path + f"/post/", out_path+f"/gif/{channel}_post.gif")
  make_gif(path + f"/beta/", out_path+f"/gif/{channel}_beta.gif")
  make_gif(path + f"/gamma/", out_path+f"/gif/{channel}_gamma.gif")


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
